In [1]:
#com unsloth para facilitar e reduzir o uso de memória/ser mais rápido

In [2]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-ctyh_uu3/unsloth_ca531e59338342eb95d661ae5bd30a81
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-ctyh_uu3/unsloth_ca531e59338342eb95d661ae5bd30a81
  Resolved https://github.com/unslothai/unsloth.git to commit 4e570be9ae4ced8cdc64e498125708e34942befc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 9.3 MB/s eta 0:00:00
   ━━━

In [3]:
#Depedências
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from transformers import TextStreamer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [4]:
max_seq_length = 2048
dtype = None
load_in_4bit = True # Quantização 4bit para reduzir o uso de memória

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [8]:
#Inferência antes de aplicar o Fine tuning

#Estrutura de dado para passar par ao modelo
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Crie uma questão de programação em Python, sobre Movimento Retilíneo Uniforme e que envolva o tópico de Decisão Simples.", # instruction
        "", # input
        "", # output - sempre em branco para receber a geração
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Crie uma questão de programação em Python, sobre Movimento Retilíneo Uniforme e que envolva o tópico de Decisão Simples.

### Input:


### Response:
```python
# Define the variables
v0 = 10 # initial velocity (m/s)
t0 = 0 # initial time (s)
a = 2 # acceleration (m/s^2)
g = 9.81 # gravity acceleration (m/s^2)

# Define the function
def motion_retilineo_uniforme(v0, t0, a, g):
    """Calculate the final velocity and position of a body
    moving with constant acceleration."""

    # Calculate the final velocity
    v1 = v0 + a * t0
    
    # Calculate the final position
    s1


In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [11]:
#Formatação e passagem de dados de treinamento
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

#local que se encontram os arquivos
localFile = "/content/sample_data/questoes.json"
print(localFile)

#carregando conjunto de dados
dataset = load_dataset("json", data_files={"train":localFile}, split = "train")
print(dataset)

#Aplicando a formatação compatível para o Llama 3.1
dataset = dataset.map(formatting_prompts_func, batched = True,)
print(dataset)

/content/sample_data/questoes.json


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 10
})


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 10
})


In [12]:
#Parâmentros de treino
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/10 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [13]:
#Realizar o Fine tuning
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 10 | Num Epochs = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.321900
2,1.259300
3,1.297600
4,1.224400
5,1.181000
6,1.064400
7,0.958200
8,0.872900
9,0.790300
10,0.659700


In [21]:
#Inferência após o Fine tuning
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Crie uma questão de programação em Python, sobre Movimento Retilíneo Uniforme e que envolva o tópico de Decisão Simples.", # instruction
        "", # input
        "", # output - sempre em branco para receber a geração
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Crie uma questão de programação em Python, sobre Movimento Retilíneo Uniforme e que envolva o tópico de Decisão Simples.

### Input:


### Response:
{Título}
Questão 1 - {Tópicos}Decisão simples{Tópicos}
{Título}

{Tema}Física - Movimento Retilíneo Uniforme{Tema}
{Introdução}
A função do movimento uniforme nos permite encontrar a posição \( S \) de um objeto que se movimenta com velocidade constante \( v \) em qualquer instante de tempo \( t \), desde que saibamos sua posição inicial \( S_i \), através da seguinte fórmula:
\[ S = S_i + v \cdot t \]
{Introdução}

{Enunciado}
Implemente um programa que receba do usuário os valores de entrada, e as armazene em variáveis: posição inicial (\( S_i \)), a velocidade (\( v \)) e o instante de tempo (\( t \)), respectivamente. Todas as entradas são número